### Initial Setup

In [2]:
import json

import pandas as pd

In [3]:
with open("data1.json") as json_file:
    data = json.load(json_file)

In [4]:
df = pd.DataFrame(data["rows"])

### Convert Dates to Datetime

In [5]:
df["created_at"] = pd.to_datetime(df["created_at"], utc=True)
df["birth_date"] = pd.to_datetime(df["birth_date"], errors="coerce", format="%B %d, %Y")
df["date_accomplished"] = pd.to_datetime(df["date_accomplished"], errors="coerce", format="%Y-%m-%d")

### Convert Monthly Income to Int

In [6]:
df["monthly_income"] = df["monthly_income"].str.replace(",", "", regex=False).str.replace(" ", "", regex=False).str.replace(".", "", regex=False)
df["monthly_income"].fillna(0, inplace=True)
df["monthly_income"] = df["monthly_income"].astype("int")

In [7]:
df["monthly_income"].head(3)

0    362307
1    407901
2    474552
Name: monthly_income, dtype: int64

### Create Age Groups Given Age

#### Calculate Age From Birth Date

In [8]:
now = pd.Timestamp.now()
df["age"] = (now - df["birth_date"]).astype("<m8[Y]")
df["age"] = df["age"].astype('Int64')

In [9]:
df["age"].head(3)

0    45
1    21
2    43
Name: age, dtype: Int64

#### Calculate Age Groups

In [10]:
bins= [0,2,4,13,20,110]
labels = ['Infant','Toddler','Kid','Teen','Adult']
df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=False)

age_group_labels = df["age_group"].value_counts().index.values.tolist()
age_group_values = df["age_group"].value_counts().values
print(df["age_group"].value_counts())
print(age_group_labels)

Adult      36
Teen       11
Infant      6
Kid         5
Toddler     2
Name: age_group, dtype: int64
['Adult', 'Teen', 'Infant', 'Kid', 'Toddler']


### Number of Citizenship per Category

In [11]:
citizenship_labels = df.groupby("citizenship").size().index.values
citizenship_values = df.groupby("citizenship").size().values
print(df.groupby("citizenship").size())

citizenship
Filipino     29
Foreigner    31
dtype: int64


### Number of Civil Status per Category

In [12]:
civil_status_labels = df.groupby("civil_status").size().index.values
civil_status_values = df.groupby("civil_status").size().values
print(df.groupby("civil_status").size())

civil_status
Divorced     13
Married      10
Separated    13
Single       11
Widowed      13
dtype: int64


### Number of Members in Family

In [13]:
house_num_labels = df["house_num"].value_counts().index.values
house_num_values = df["house_num"].value_counts().values
print(df["house_num"].value_counts())

541253    9
779223    8
808607    8
848962    7
901399    7
100273    6
867936    5
222218    4
316853    3
575499    1
587108    1
557162    1
Name: house_num, dtype: int64


### Average Family Members in A house

In [14]:
house_members_avg = df["house_num"].value_counts().mean().astype(int)

### Number of Social Classes per Person

In [15]:
bins = [0, 7890, 15780, 31560, 78900, 118350, 157800, 200000, 500000]
labels = ["Not Applicable","Poor", "Low Income", "Lower Middle Income", "Middle Middle Income", "Upper Middle Income", "Upper Income", "Rich"]
df["social_class"] = pd.cut(df["monthly_income"], bins=bins, labels=labels, right=False)

social_class_labels = df["social_class"].value_counts().index.categories.values
social_class_values = df["social_class"].value_counts().values
print(df["social_class"].value_counts())

Rich                    28
Not Applicable          22
Upper Middle Income      4
Lower Middle Income      2
Upper Income             2
Poor                     1
Middle Middle Income     1
Low Income               0
Name: social_class, dtype: int64


### Number of RBI in Database

In [24]:
len(df.groupby("house_num").sum().index.values.tolist())

12

### Average Salary per Person

In [ ]:
person_avg_salary = df[df["monthly_income"] != 0]["monthly_income"].mean().astype("int")

### Average Salary per Household

In [ ]:
household = df.groupby(["house_num"])["monthly_income"].sum()
household_avg_salary = household.values.mean().astype("int")

In [ ]:
df